In [59]:
import random
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import pandas as pd
from nn_edm import *
from gru_nn.py import *

ModuleNotFoundError: No module named 'gru_nn.py'; 'gru_nn' is not a package

In [55]:
'''
Binary prediction skill (“bloom” or “no bloom”) is
evaluated by how well bloom predictions above a defined threshold
correspond to bloom observations above the same threshold
Input: Series of observed chlorophyll-a, series of predicted chlorophyll-1
Output: Arrray containing [Accuracy, True Positive Rate, False Positive Rate, True Negative Rate, False Negative Rate]
'''
def thresh_bloom_binary_prediction(obs,pred,threshold=8.03199999999999):
    obs_blooms = obs > threshold
    pred_blooms = pred > threshold
    Accuracy = 1 - (obs_blooms ^ pred_blooms).mean()
    True_pos = (obs_blooms & pred_blooms).sum() / obs_blooms.sum()
    False_pos = ((~obs_blooms) & pred_blooms).sum() / (~obs_blooms).sum()
    True_neg = ((~obs_blooms) & (~pred_blooms)).sum() / (~obs_blooms).sum()
    False_neg = (obs_blooms & (~pred_blooms)).sum() / obs_blooms.sum()
    
    return [Accuracy, True_pos, False_pos, True_neg, False_neg]

In [56]:
'''
Binary prediction skill (“bloom” or “no bloom”) is
evaluated by how well bloom predictions (5% largest predicted values)
correspond to bloom observations (5%largest observed values)
Input: Series of observed chlorophyll-a, series of predicted chlorophyll-1
Output: Arrray containing [Accuracy, True Positive Rate, False Positive Rate, True Negative Rate, False Negative Rate]
'''
def bloom_binary_prediction(obs,pred):
    obs_bloom_95 = np.percentile(obs, 95) #incorrect
    pred_bloom_95 = np.percentile(pred, 95) #incorrect
    obs_blooms = obs > obs_bloom_95
    pred_blooms = pred > pred_bloom_95
    Accuracy = 1 - (obs_blooms ^ pred_blooms).mean()
    True_pos = (obs_blooms & pred_blooms).sum() / obs_blooms.sum()
    False_pos = ((~obs_blooms) & pred_blooms).sum() / (~obs_blooms).sum()
    True_neg = ((~obs_blooms) & (~pred_blooms)).sum() / (~obs_blooms).sum()
    False_neg = (obs_blooms & (~pred_blooms)).sum() / obs_blooms.sum()
    
    return [Accuracy, True_pos, False_pos, True_neg, False_neg]

In [57]:
#Load embedding data
input_file_path = '../Data/cleaned_data.csv'
target = 'Avg_Chloro'
data = pd.read_csv(input_file_path)
data = data.set_index('time (UTC)')
data['Time'] = data.index.astype(int)
data = data.drop(columns=['Time'])

tau_lengths = [-1,-2,-3]
E = 6
X, y = get_data(data, E, tau_lengths, target=target)
embd_sz = len(data.columns) * E * len(tau_lengths)
X = torch.tensor(X[533:], dtype=torch.float) 
y = torch.tensor(y[533:], dtype=torch.float)
len(y)

283

In [58]:
#model = NNEDMModel(embd_sz,hidden_size=100)
model = G
model.load_state_dict(torch.load("model.pth"))

model.eval()
prediction = []
for inp in X:
    prediction.append(model(inp))
prediction = torch.tensor(prediction).numpy()

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_27790/2395043709.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth")

RuntimeError: Error(s) in loading state_dict for NNEDMModel:
	Missing key(s) in state_dict: "fw.weight", "fw.bias", "output.weight", "output.bias". 
	Unexpected key(s) in state_dict: "gru.weight_ih_l0", "gru.weight_hh_l0", "gru.bias_ih_l0", "gru.bias_hh_l0", "gru.weight_ih_l1", "gru.weight_hh_l1", "gru.bias_ih_l1", "gru.bias_hh_l1", "attn.in_proj_weight", "attn.in_proj_bias", "attn.out_proj.weight", "attn.out_proj.bias", "fc.weight", "fc.bias". 

In [52]:
y.numpy()

array([  0.7 ,   0.67,   1.41,   1.48,   0.97,   1.56,   0.74,   1.97,
         1.49,   2.26,   1.39,   1.54,   2.3 ,   2.01,   2.7 ,   0.45,
         1.43,   0.91,   2.21,   8.54,   5.82,  55.68,   6.28,  16.68,
         4.29,   1.86,   0.94,   3.26,   1.74,   1.92,   1.84,   6.26,
         2.69,   6.8 ,   4.42,   2.08,   1.78,   2.17,   2.36,   2.17,
         1.36,   0.16,   0.98,   0.84,   0.94,   1.28,   1.43,   2.42,
         1.28,   2.39,   1.49,   1.63,   1.11,   1.12,   1.42,   0.57,
         0.8 ,   0.99,   0.9 ,   0.84,   0.83,   0.72,   1.73,   0.86,
         2.71,   2.71,   2.71,   2.71,   2.71,   2.71,   2.71,   2.71,
         2.71,   4.56,   1.6 ,   2.35,   2.54,   2.08,   2.44,   3.29,
         2.79,   3.86,   1.59,   3.66,   0.93,   2.12,   1.66,   1.6 ,
         1.58,   1.76,   1.83,   1.58,   1.78,   1.61,   1.61,   1.45,
         2.02,   4.76,   2.93,   1.54,   1.5 ,   1.92,   2.88,   2.9 ,
         3.48,   4.22,   5.44,   2.47,   4.11,   7.96,   3.23,   2.46,
      

In [53]:
bloom_binary_prediction(y.numpy(),prediction[:-1])

[np.float64(0.9151943462897526),
 np.float64(0.2),
 np.float64(0.04477611940298507),
 np.float64(0.9552238805970149),
 np.float64(0.8)]